# Car Quality Metrics

## Estimate the Quality using the Loaded Classifier

In [1]:
import os
import glob
import PIL.Image
import numpy as np
import torch
from car_quality_estimator.car_quality_metric import load_car_quality_score

test_image_dir = "example_data/2c21b97ff3dc4fc3b1ef9e4bb0164318"
car_quality_metric = load_car_quality_score(use_combined_embedding_model=True)

Using models from package installation directory: /mnt/damian/Projects/quality_assessment_library/car_quality_estimator/models


/mnt/damian/Projects/quality_assessment_library/car_quality_estimator/individual_image_classifier.py:393: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state

In [17]:
# load all images from the test directory and store the PIL images in a list
all_images = glob.glob(os.path.join(test_image_dir, "*.png"))
all_images = [PIL.Image.open(image).convert("RGB") for image in all_images]
reference_batch = all_images

# let's generate some distorted views using Gaussian noise
def create_noised_image(image_list):
    all_images_np = np.array(image_list) / 255.0
    all_images_distorted_np  = all_images_np + np.random.randn(*all_images_np.shape) * 0.0055
    all_images_distorted_np = np.clip(all_images_distorted_np, 0, 1)
    all_images_distorted_pil = [PIL.Image.fromarray((img * 255).astype(np.uint8)) for img in all_images_distorted_np]
    return all_images_distorted_pil
generated_views = create_noised_image(reference_batch)

In [7]:
# Quality score with reference images
car_quality_metric.compute_scores_no_reference(generated_views)

{'avg_quality_score': 0.18115516,
 'avg_entropy': 0.062057108,
 'avg_combined_score': 0.16658795,
 'quality_std': 0.36009538,
 'num_samples': 21}

In [18]:
# Quality score with reference images
car_quality_metric.compare_with_reference(
    generated_views, reference_batch, compute_kid=True)

{'generated_metrics': {'avg_quality_score': 0.23797481,
  'avg_entropy': 0.11658727,
  'avg_combined_score': 0.2111067,
  'quality_std': 0.38184297,
  'num_samples': 21},
 'reference_metrics': {'avg_quality_score': 0.22754328,
  'avg_entropy': 0.0401953,
  'avg_combined_score': 0.21761395,
  'quality_std': 0.40535438,
  'num_samples': 21},
 'quality_gap': -0.010431528,
 'score_distribution_metrics': {'kl_divergence_kde': -0.04320147668210633,
  'jensen_shannon_distance': 0.24816035841395828,
  'wasserstein_distance': 0.033203934826216204},
 'kid_metrics': {'kid_score': 6415.784951636902,
  'n_gen_samples': 21,
  'n_ref_samples': 21}}